# UN ENVIRONMENTAL DATA ANALYSIS

### 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sqlalchemy import create_engine
from matplotlib.ticker import FuncFormatter

import os
from dotenv import load_dotenv

SyntaxError: invalid syntax (3781310313.py, line 2)

In [3]:
load_dotenv()

#Everything is put into a .env file
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')

engine = create_engine(f'mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')

### Queries
After our imports we will begin by bringing in our SQL queries.


In [4]:
global_co2emissions = '''
SELECT * FROM global_co2emissions;
'''

global_threatened_species = '''
SELECT * FROM global_threatened_species;
'''

global_forest_cover = '''
SELECT * FROM global_forest_cover;
'''

global_permanent_crops = '''
SELECT * FROM global_permanent_crops;
'''

joined_co2_ts = '''
SELECT * FROM joined_co2_ts;
'''

joined_co2_fc = '''
SELECT * FROM joined_co2_fc;
'''

joined_ts_pc = '''
SELECT * FROM joined_ts_pc;
'''

joined_ts_fc = '''
SELECT * FROM joined_ts_fc;
'''

Now we will set each of the values within the data frames to have three decimale places. 

In [5]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Here are a few dataframes from the SQL queries.

In [6]:
global_co2emissions_df = pd.read_sql(global_co2emissions, engine)
#I wanted to make the numbers more readable, so I changed the dataframe values to be metric tons instead of thousand metric tons.
global_co2emissions_df['total_emissions'] = global_co2emissions_df['total_emissions'] * 1000
global_co2emissions_df['Series'] = "Emissions (metric tons of carbon dioxide)"
global_co2emissions_df.head(10)

DatabaseError: (mysql.connector.errors.DatabaseError) 2005 (HY000): Unknown MySQL server host 'None' (11001)
(Background on this error at: https://sqlalche.me/e/20/4xp6)

In [ ]:
global_co2emissions_df.describe().drop(['year'], axis=1)

In [ ]:
global_threatened_species_df = pd.read_sql(global_threatened_species, engine)
global_threatened_species_df.head(5)

In [ ]:
global_threatened_species_df.describe().drop(['year'], axis=1)

In [ ]:
global_forest_cover_df = pd.read_sql(global_forest_cover, engine)
global_forest_cover_df.head(10)

In [ ]:
global_forest_cover_df.describe().drop(['year'], axis=1)

In [ ]:
global_permanent_crops_df = pd.read_sql(global_permanent_crops, engine)
global_permanent_crops_df.head(10)

In [ ]:
global_permanent_crops_df.describe().drop(['year'], axis=1)

We can look at some of the basic statistics of these single sets. 

In [ ]:
globEmissAvg = global_co2emissions_df['total_emissions'].mean()
globEmissAvg = pd.to_numeric(globEmissAvg)
print("Global Emissions Average:", int(abs(globEmissAvg)), "metric tons of Carbon Dioxide.")
globEmissRange = int(global_co2emissions_df['total_emissions'].max()) - int(global_co2emissions_df['total_emissions'].min())
print("Global Emissions increased by", globEmissRange, "metric tons of Carbon Dioxide from 1975 to 2017.")

Below are the joined SQL dataframes.

In [ ]:
joined_co2_ts_df = pd.read_sql(joined_co2_ts, engine)
joined_co2_fc_df = pd.read_sql(joined_co2_fc, engine)
joined_ts_pc_df = pd.read_sql(joined_ts_pc, engine)
joined_ts_fc_df = pd.read_sql(joined_ts_fc, engine)

Were going to have to convert each of the joined dataframes that have CO2 data in them to metric tons. 

In [ ]:
joined_co2_ts_df['CO2 Value'] = joined_co2_ts_df['CO2 Value'] * 1000
joined_co2_ts_df['CO2 Series'] = "Emissions (metric tons of carbon dioxide)"

In [ ]:
joined_co2_fc_df['CO2 Value'] = joined_co2_fc_df['CO2 Value'] * 1000
joined_co2_fc_df['CO2 Series'] = "Emissions (metric tons of carbon dioxide)"

Within each of the joined dataframes there is a year column for each added dataset. I will create a 'Year' column that has each year of data for both sets. Below is an example dataframe. Notice on the right the 'Year' Column. 

In [ ]:
joined_co2_ts_df['CO2 Year'] = pd.to_numeric(joined_co2_ts_df['CO2 Year'], errors='coerce')
joined_co2_ts_df['TS Year'] = pd.to_numeric(joined_co2_ts_df['TS Year'], errors='coerce')

#here we are creating the year column that joins all years for 'CO2 Year' and 'TS Year'
joined_co2_ts_df['Year'] = joined_co2_ts_df['CO2 Year'].combine_first(joined_co2_ts_df['TS Year'])

#turning it from a float to an int
joined_co2_ts_df['Year'] = joined_co2_ts_df['Year'].astype(int)

#sorting in numerical order by the new Year column
joined_co2_ts_df.sort_values(by='Year', ascending=True, na_position='last', inplace=True)

In [ ]:
#Here is the dataframe without each of the 'Year' columns we are no longer using.
joined_co2_ts_df[['CO2 Country/Region', #'CO2 Year', 
                  'CO2 Series', 'CO2 Value',
                   'TS Country/Region',
                  #'TS Year', 
                  'TS Series', 'TS Value', 'Year']]

In [ ]:
joined_co2_fc_df['CO2 Year'] = pd.to_numeric(joined_co2_fc_df['CO2 Year'], errors='coerce')
joined_co2_fc_df['FC Year'] = pd.to_numeric(joined_co2_fc_df['FC Year'], errors='coerce')

joined_co2_fc_df['Year'] = joined_co2_fc_df['CO2 Year'].combine_first(joined_co2_fc_df['FC Year'])

joined_co2_fc_df['Year'] = joined_co2_fc_df['Year'].astype(int)

joined_co2_fc_df.sort_values(by='Year', ascending=True, na_position='last', inplace=True)

In [ ]:
joined_ts_pc_df['TS Year'] = pd.to_numeric(joined_ts_pc_df['TS Year'], errors='coerce')
joined_ts_pc_df['PC Year'] = pd.to_numeric(joined_ts_pc_df['PC Year'], errors='coerce')

joined_ts_pc_df['Year'] = joined_ts_pc_df['TS Year'].combine_first(joined_ts_pc_df['PC Year'])

joined_ts_pc_df['Year'] = joined_ts_pc_df['Year'].astype(int)

joined_ts_pc_df.sort_values(by='Year', ascending=True, na_position='last', inplace=True)

In [ ]:
joined_ts_fc_df['TS Year'] = pd.to_numeric(joined_ts_fc_df['TS Year'], errors='coerce')
joined_ts_fc_df['FC Year'] = pd.to_numeric(joined_ts_fc_df['FC Year'], errors='coerce')

joined_ts_fc_df['Year'] = joined_ts_fc_df['TS Year'].combine_first(joined_ts_fc_df['FC Year'])

joined_ts_fc_df['Year'] = joined_ts_fc_df['Year'].astype(int)

joined_ts_fc_df.sort_values(by='Year', ascending=True, na_position='last', inplace=True)

In [ ]:
#First, we will make the values of the emissions equal to million metric tons rather than thousand metric tons for readability
global_co2emissions_df['total_emissions_millions'] = global_co2emissions_df['total_emissions'] / 1_000_000

### Single Set Visualizations 

Next we will start on some visualizations on the single set dataframes.

In [ ]:
#visualizing the co2 emissions for each year in our dataframe.

plt.plot(global_co2emissions_df['year'], global_co2emissions_df['total_emissions_millions'], marker='o', label='CO2 Emissions')
plt.xlabel('Year')
plt.ylabel('Emissions (million metric tons of carbon dioxide)')
plt.title('Global CO2 Emissions Over Time')
plt.xticks(ticks=range(global_co2emissions_df['year'].min(), global_co2emissions_df['year'].max()+1, 2), rotation=45)
plt.legend()
plt.show()

As we can see, from 1975 (the first year from our dataframe) to 2017 (the final year in our dataframe), the amount of CO2 emissions has increased greatly.

In [ ]:
#Next, let us visualize the total global threatened species number. 
#This includes plants, invertebrates, and vertebrates.
#For this, I will be using Plotly to explore interactive visualization within this project.

ts_fig = px.line(global_threatened_species_df,
                x='year',
                y='total_threatened_species',
                markers=True,
                title='Global Number of Threatened Species',
                labels={'total_threatened_species' : 'Total Number of Threatened Species', 'year' : 'Year'})
ts_fig.update_traces(hovertemplate='Total: %{y:.0f}<br>Year: %{x}')
ts_fig.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(global_forest_cover_df['year'], height=global_forest_cover_df['value'], color='g')
plt.xlabel('Years')
plt.ylabel('Forest Cover (thousand hectares)')
plt.title('Forest Cover (thousand hectares) Over Time')
plt.xticks(ticks=range(global_forest_cover_df['year'].min(), global_forest_cover_df['year'].max() + 1, 1))
y_ticks = [3990000, 4000000, 4010000, 4020000, 4030000, 4040000]
plt.ylim(3980000, 4050000) 

def millions(x, pos):
    return '%d' % x

plt.gca().yaxis.set_major_formatter(FuncFormatter(millions))
plt.gca().set_yticks(y_ticks)

plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(global_permanent_crops_df['year'], height=global_permanent_crops_df['value'], color='gold')
plt.xlabel('Years')
plt.ylabel('Permanent Crops (thousand hectares)')
plt.title('Permanent Crops (thousand hectares) Over Time')
plt.xticks(ticks=range(global_permanent_crops_df['year'].min(), global_permanent_crops_df['year'].max() + 1, 1))
y_ticks = [130000, 135000, 140000, 145000, 150000, 155000, 160000, 165000, 170000]
plt.ylim(130000, 170000) 

def crops(x, pos):
    return '%d' % x

plt.gca().yaxis.set_major_formatter(FuncFormatter(crops))
plt.gca().set_yticks(y_ticks)

plt.show()

### Joined Set Visualizations

Now we will begin to visualize each of the joined dataframes.

In [ ]:
joined_co2_ts_df.head(10)

Notice we have NaN for certain years. In each dataset, the years that data was collected differs. For this dataframe in specific we will visualize it to include the NaN values then we will use linear interpolation to visualize interpolated values in the NaN slots. 

In [ ]:
#visualizing the co2 emissions / threatened species dataframe with NaN values.
fig, ax1 = plt.subplots(figsize=(10,6))

#left y-axis
ax1.plot(joined_co2_ts_df['Year'], joined_co2_ts_df['CO2 Value'], marker='o', color='b', label='CO2 Emissions')
ax1.set_xlabel("Year")
ax1.set_ylabel("Emissions (metric tons of carbon dioxide)")
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_ylim(0, 40000000000)
ax1.set_yticks([10e9, 15e9, 20e9, 25e9, 30e9, 35e9])
def billion_formatter(x, pos):
    return f'{x / 1e9:.2f} Billion'

# for readability, I have to use FuncFormatter to format the y-axis.
ax1.yaxis.set_major_formatter(FuncFormatter(billion_formatter))

#Right y-axis
ax2 = ax1.twinx()
ax2.plot(joined_co2_ts_df['Year'], joined_co2_ts_df['TS Value'], marker='o', color='r', label='Threatened Species')
ax2.set_ylabel("Number of Threatened Species")
ax2.tick_params(axis='y', labelcolor='r')

plt.title("CO2 Emissions & Threatened Species")
plt.xticks(ticks=range(joined_co2_ts_df['Year'].min(), joined_co2_ts_df['Year'].max()+1, 3), rotation=45)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()

Now let's try using interpolation for the missing values.

In [ ]:
joined_co2_ts_df['CO2 Value'] = joined_co2_ts_df['CO2 Value'].interpolate()
joined_co2_ts_df['TS Value'] = joined_co2_ts_df['TS Value'].interpolate()
joined_co2_ts_df.head(10)

Now the interpolated visualization.

In [ ]:
#visualizing the co2 emissions / threatened species dataframe with interpolated values.
fig, ax1 = plt.subplots(figsize=(10,6))

ax1.plot(joined_co2_ts_df['Year'], joined_co2_ts_df['CO2 Value'], marker='o', color='b', label='CO2 Emissions')
ax1.set_xlabel("Year")
ax1.set_ylabel("Emissions (metric tons of carbon dioxide)")
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_ylim(0, 40000000000)
ax1.set_yticks([10e9, 15e9, 20e9, 25e9, 30e9, 35e9])
def billion_formatter(x, pos):
    return f'{x / 1e9:.2f} Billion'
ax1.yaxis.set_major_formatter(FuncFormatter(billion_formatter))

ax2 = ax1.twinx()
ax2.plot(joined_co2_ts_df['Year'], joined_co2_ts_df['TS Value'], marker='o', color='r', label='Threatened Species')
ax2.set_ylabel("Number of Threatened Species")
ax2.tick_params(axis='y', labelcolor='r')

plt.title("CO2 Emissions & Threatened Species")
plt.xticks(ticks=range(joined_co2_ts_df['Year'].min(), joined_co2_ts_df['Year'].max()+1, 3), rotation=45)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()

We will now do the CO2 Emissions and Forest Cover plots. I will be using interpolated data.

In [ ]:
joined_co2_fc_df['CO2 Value'] = joined_co2_fc_df['CO2 Value'].interpolate()
joined_co2_fc_df['FC Value'] = joined_co2_fc_df['FC Value'].interpolate()
joined_co2_fc_df.head(10)

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,6))


ax1.bar(joined_co2_fc_df['Year'], joined_co2_fc_df['CO2 Value'], color='cornflowerblue', label='CO2 Emissions')
ax1.set_xlabel("Year")
ax1.set_ylabel("Emissions (metric tons of carbon dioxide)", color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_ylim(0, 40000000000)
ax1.set_yticks([10e9, 15e9, 20e9, 25e9, 30e9, 35e9])


def billion_formatter(x, pos):
    return f'{x / 1e9:.2f} Billion'
ax1.yaxis.set_major_formatter(FuncFormatter(billion_formatter))


ax2 = ax1.twinx()
ax2.scatter(joined_co2_fc_df['Year'], joined_co2_fc_df['FC Value'], 
            color='olivedrab', edgecolor='black', s=100, alpha=0.8, label='Forest Cover', marker='o')
ax2.set_ylabel("Forest Cover (thousand hectares)", color='g')
ax2.tick_params(axis='y', labelcolor='g')

plt.title("CO2 Emissions & Forest Cover Over Time")
plt.xticks(ticks=range(joined_co2_fc_df['Year'].min(), joined_co2_fc_df['Year'].max()+1, 3), rotation=45)

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
joined_ts_pc_df['TS Value'] = joined_ts_pc_df['TS Value'].interpolate()
joined_ts_pc_df['PC Value'] = joined_ts_pc_df['PC Value'].interpolate()
joined_ts_pc_df.head(10)

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,6))

# Plot 1: Line plot for Threatened Species (solid line)
ax1.plot(joined_ts_pc_df['Year'], joined_ts_pc_df['TS Value'], color='black', linestyle='-', marker='o', label='Threatened Species')
ax1.set_xlabel("Year")
ax1.set_ylabel("Threatened Species (Total number)", color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.set_title("Threatened Species and Permanent Crops Over Time")
def species_formatter(x, pos):
    return f'{x:,.0f}'
ax1.yaxis.set_major_formatter(FuncFormatter(species_formatter))

ax2 = ax1.twinx()
ax2.plot(joined_ts_pc_df['Year'], joined_ts_pc_df['PC Value'], color='yellowgreen', linestyle='--', marker='x', label='Permanent Crops')
ax2.set_ylabel("Permanent Crops (thousand hectares)", color='yellowgreen')
ax2.tick_params(axis='y', labelcolor='yellowgreen')

def crop_formatter(x, pos):
    return f'{x:,.0f}K'
ax2.yaxis.set_major_formatter(FuncFormatter(crop_formatter))

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
joined_ts_fc_df['TS Value'] = joined_ts_fc_df['TS Value'].interpolate()
joined_ts_fc_df['FC Value'] = joined_ts_fc_df['FC Value'].interpolate()
joined_ts_fc_df.head(10)

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

barWidth = 0.4
x = np.arange(len(joined_ts_fc_df)) 

ax1.bar(x - barWidth/2, joined_ts_fc_df['TS Value'], width=barWidth, color='r', alpha=0.6, label='Threatened Species')
ax1.set_xlabel("Categories")
ax1.set_ylabel("Threatened Species", color='r')
ax1.tick_params(axis='y', labelcolor='r')
def formatter1(x, pos):
    return f'{int(x):,}'
ax1.yaxis.set_major_formatter(FuncFormatter(formatter1))

ax2 = ax1.twinx()
ax2.bar(x + barWidth/2, joined_ts_fc_df['FC Value'] / 1000, width=barWidth, color='g', alpha=0.6, label='Forest Cover')
ax2.set_ylabel("Forest Cover (thousand hectares)", color='g')
ax2.tick_params(axis='y', labelcolor='g')
def formatter2(x, pos):
    return f'{int(x):,}'
ax2.yaxis.set_major_formatter(FuncFormatter(formatter2))

ax1.set_xticks(x)
ax1.set_xticklabels(joined_ts_fc_df['Year'].astype(str), rotation=45)
ax2.set_ylim(3900, 4100)

plt.title("Bar Chart of Threatened Species and Forest Cover")
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()
plt.show()